In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

In [ ]:
# loading the dataset to a pandas DataFrame
comments_dataset = pd.read_csv('/content/choridata.csv')

In [ ]:
comments_dataset.shape

In [ ]:
comments_dataset.head()

In [ ]:
comments_dataset.isnull().sum()

In [ ]:
comments_dataset[comments_dataset['TextBlobAnalysis'] == 'Neutral'].shape[0]

In [ ]:
comments_dataset[comments_dataset['TextBlobAnalysis'] == 'Positive'].shape[0]

In [ ]:
comments_dataset[comments_dataset['TextBlobAnalysis'] == 'Negative'].shape[0]

In [ ]:
comments_dataset[comments_dataset['VaderAnalysis'] == 'Neutral'].shape[0]

In [ ]:
comments_dataset[comments_dataset['VaderAnalysis'] == 'Positive'].shape[0]

In [ ]:
comments_dataset[comments_dataset['VaderAnalysis'] == 'Negative'].shape[0]

In [ ]:
comments_amb = comments_dataset[comments_dataset['VaderAnalysis'] != comments_dataset['TextBlobAnalysis']]

In [ ]:
comments_amb

In [ ]:
comments_vader = comments_dataset[['Comments', 'VaderAnalysis']]

In [ ]:
comments_vader

In [ ]:
comments_vader['VaderAnalysis'].value_counts()


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
comments_vader['VaderAnalysis'] = le.fit_transform(comments_vader['VaderAnalysis'])

print("negative = 0 \nneutral = 1 \npositive = 2")

In [ ]:
comments_vader = comments_vader.sample(10000)

In [ ]:
# !python -m pip install tensorflow_text


In [ ]:
# !pip install -q -U "tensorflow-text==2.8.*"


In [ ]:
# !pip install tensorflow-text

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.metrics.pairwise import cosine_similarity


from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(comments_vader['Comments'], comments_vader['VaderAnalysis'], test_size = 0.3, random_state = 5, stratify =comments_vader['VaderAnalysis'])

In [ ]:
encoder_url  = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'
preprocess_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [ ]:
bert_preprocess_model = hub.KerasLayer(preprocess_url)
bert_encoder = hub.KerasLayer(encoder_url)

In [ ]:
def get_embeddings(sentence):
  preprocessed_text = encoder_url(sentence)
  embedding = preprocess_url(preprocessed_text)['pooled_output']

In [ ]:
# Bert Layers
input_layer = tf.keras.layers.Input(shape = (),dtype = tf.string, name = 'Input')
preprocessed_text = bert_preprocess_model(input_layer)
output =  bert_encoder(preprocessed_text)

In [ ]:
output_1 = tf.keras.layers.Dropout(0.1, name = 'dropout')(output['pooled_output'])
output_cat = tf.keras.layers.Dense(1, activation = 'softmax', name = 'category')(output_1)

In [ ]:
model =tf.keras.Model(inputs = [input_layer], outputs = [output_cat])

In [ ]:
model.summary()

In [ ]:
tf.keras.optimizers.Adam(
    learning_rate=0.1,
    name = 'adam'

)

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs =5)

In [ ]:
model.evaluate(x_test, y_test)